In [125]:
import yfinance as yf
import pandas as pd

In [24]:
stz_ticker = yf.Ticker('STZ')

In [90]:
income_statement = stz_ticker.get_income_stmt()

In [132]:
def get_bronze_tables(ticker, financial_statement_report):
    current_ticker = yf.Ticker(ticker)
    if financial_statement_report == 'bs':
        return current_ticker.get_balance_sheet()
    elif financial_statement_report == 'pl':
        return current_ticker.get_income_stmt()
    elif financial_statement_report == 'cf':
        return current_ticker.get_cashflow()
    else: return None

In [ ]:
def get_raw_yahoo_financial_statements(ticker):
    

In [121]:
unpivot_income_statement = income_statement.\
    reset_index().\
    melt(id_vars='index', var_name='date', value_name='value').\
    rename(columns={'index':'metric'})

In [124]:
unpivot_income_statement['date'] =  pd.to_datetime(unpivot_income_statement['date'])

In [128]:
unpivot_income_statement['value'] = unpivot_income_statement['value'].round(0)

In [129]:
unpivot_income_statement

,metric,date,value
0,TaxEffectOfUnusualItems,2025-02-28,-6.424110e+08
1,TaxRateForCalcs,2025-02-28,0.000000e+00
2,NormalizedEBITDA,2025-02-28,3.833400e+09
3,TotalUnusualItems,2025-02-28,-3.059100e+09
4,TotalUnusualItemsExcludingGoodwill,2025-02-28,-3.059100e+09
...,...,...,...
230,GrossProfit,2021-02-28,NaN
231,CostOfRevenue,2021-02-28,NaN
232,TotalRevenue,2021-02-28,NaN
233,ExciseTaxes,2021-02-28,NaN
